In [2]:
import pandas as pd
import numpy as np

### 1.获取上市公司和子公司所在城市

#### 1.1数据读取

In [5]:
df_com = pd.read_excel(r'../../file/subject/chapter3/SSGScompany04.xlsx',sheet_name='Sheet1')

In [7]:
df_com.head(1)

,Unnamed: 0,Unnamed: 0.1,公司名称,地址,公司网址,爬取页,公司名称2,region,province,city
0,0,1781,安徽华茂纺织股份有限公司,安徽省安庆市纺织南路80号,/orgcompany/searchitemdtl/5bc2e76fd05b73a3139c...,4,安徽华茂纺织股份有限公司,"['安徽省', '安庆市', '大观区']",安徽省,安庆市


In [9]:
df_son = pd.read_excel(r'../../file/subject/chapter3/company_info_all05.xlsx',sheet_name='Sheet1')

In [10]:
df_son.head(1)

,Unnamed: 0,公司名称,上市公司名称,公司状态,注册资本,上市公司投资比例,province,city
0,0,上海盛美投资咨询有限公司,甘肃亚盛实业(集团)股份有限公司,存续（在营、开业、在册）,500.0000(万人民币),80%,上海市,上海市


#### 1.2数据清洗

In [12]:
#将上市公司和子公司数据链接
df = pd.merge(df_son,df_com,left_on='上市公司名称',right_on='公司名称2').reset_index()

In [16]:
#判断子公司状态，空值设置为开业
df.公司状态 = df.公司状态.apply(lambda x:'开业' if x == 0 else x)

In [17]:
len(df)

57262

In [20]:
#筛选掉吊销状态的公司
len(df[df.公司状态.str.contains('吊销')])

938

In [21]:
df = df[-df.公司状态.str.contains('吊销')]
len(df)

56324

In [23]:
#获取上市公司和子公司所在城市
city_net = df[['city_y','city_x']]

In [25]:
city_net.head()

,city_y,city_x
0,兰州市,上海市
3,兰州市,兰州市
6,兰州市,兰州市
9,兰州市,张掖市
10,兰州市,兰州市


### 2.统计各城市上市公司和子公司对应关系

In [26]:
city_net['count'] = 1

C:\Users\cc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
city_net.head()

,city_y,city_x,count
0,兰州市,上海市,1
3,兰州市,兰州市,1
6,兰州市,兰州市,1
9,兰州市,张掖市,1
10,兰州市,兰州市,1


In [32]:
len(city_net)

56324

#### 2.1去掉上市公司和子公司同城的数据

In [36]:
city_net = city_net[-(city_net['city_y']==city_net['city_x'])]

In [37]:
len(city_net)

33809

#### 2.2聚合上市公司和子公司所在城市，得到对应数据

In [45]:
city_net.groupby(by=['city_y','city_x']).sum().reset_index()

,city_y,city_x,count
0,七台河市,北京市,1
1,七台河市,双鸭山市,2
2,七台河市,鸡西市,1
3,三亚市,上海市,5
4,三亚市,九江市,1
...,...,...,...
8134,龙岩市,遵义市,1
8135,龙岩市,长春市,1
8136,龙岩市,阿坝藏族羌族自治州,1
8137,龙岩市,陇南市,1


In [46]:
city_net.groupby(by=['city_y','city_x']).sum().reset_index().sort_values(by='count',ascending=False)

,city_y,city_x,count
1142,北京市,深圳市,885
958,北京市,上海市,423
182,上海市,深圳市,270
1032,北京市,天津市,239
5285,深圳市,北京市,224
...,...,...,...
3708,成都市,绥化市,1
3707,成都市,绍兴市,1
3706,成都市,红河哈尼族彝族自治州,1
3705,成都市,秦皇岛市,1


### 3.可视化

In [51]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
author_lst = []
for i in author_use_sum['from']:
    author_lst.append(i)
for i in author_use_sum['to']:
    author_lst.append(i) 

author_dic = {}
nodelst = []
sizes = []
weights = []
for i in author_lst:
    author_dic[i] = author_lst.count(i)
for k,v in author_dic.items():
    nodelst.append(k)
    sizes.append(v*1000)
    weights.append(v*2)

labels = {}
for k,v in author_dic.items():
    if v >= 3:
        labels[k] = k

In [ ]:
GA = nx.from_pandas_edgelist(author_use_sum.head(100),source='from',target='to',edge_attr='weight')
plt.figure(figsize=(25,25))
pos = nx.spring_layout(GA,k=0.25)

nx.draw(GA,pos,nodelist=nodelst,node_size=sizes,alpha=0.6)
nx.draw_networkx_labels(GA,pos,labels,font_family='SimHei',font_size=20,font_color='black')
nx.draw_networkx_edges(GA,pos,width=weights,alpha=0.5)
plt.savefig('../file/专题一network.png')